# Mandatory Challenge
## Context
You work in the data analysis team of a very important company. On Monday, the company shares some good news with you: you just got hired by a major retail company! So, let's get prepared for a huge amount of work!

Then you get to work with your team and define the following tasks to perform:   
1. You need to start your analysis using data from the past.  
2. You need to define a process that takes your daily data as an input and integrates it.  

You are in charge of the second part, so you are provided with a sample file that you will have to read daily. To complete you task, you need the following aggregates:
* One aggregate per store that adds up the rest of the values.
* One aggregate per item that adds up the rest of the values.

You can import the dataset `warehouse_and_retail_sales` from Ironhack's database. 

## Your task
Therefore, your process will consist of the following steps:
1. Read the sample file that a daily process will save in your folder. 
2. Clean up the data.
3. Create the aggregates.
4. Write three tables in your local database: 
    - A table for the cleaned data.
    - A table for the aggregate per supplier.
    - A table for the aggregate per item.

## Instructions
* Read the csv you can find in Ironhack's database.
* Clean the data and create the aggregates as you consider.
* Create the tables in your local database.
* Populate them with your process.

In [2]:
import pandas as pd

In [3]:
warehouse = pd.read_csv(r"C:\Users\hp\Documents\M1\lab_df_calculations-main_X\lab_df_calculations-main\your-code\warehouse.csv")
warehouse

,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
0,2020,1,REPUBLIC NATIONAL DISTRIBUTING CO,100009,BOOTLEG RED - 750ML,WINE,0.00,0.0,2.00
1,2020,1,PWSWN INC,100024,MOMENT DE PLAISIR - 750ML,WINE,0.00,1.0,4.00
2,2020,1,RELIABLE CHURCHILL LLLP,1001,S SMITH ORGANIC PEAR CIDER - 18.7OZ,BEER,0.00,0.0,1.00
3,2020,1,LANTERNA DISTRIBUTORS INC,100145,SCHLINK HAUS KABINETT - 750ML,WINE,0.00,0.0,1.00
4,2020,1,DIONYSOS IMPORTS INC,100293,SANTORINI GAVALA WHITE - 750ML,WINE,0.82,0.0,0.00
...,...,...,...,...,...,...,...,...,...
307640,2020,9,DOPS INC,97896,ST PETERS ORGANIC ENG ALE NR 12/CS - 16.9OZ,BEER,0.00,0.0,1.00
307641,2020,9,ANHEUSER BUSCH INC,97918,STELLA ARTOIS 2/12 NR - 11.2OZ,BEER,372.45,315.0,3586.88
307642,2020,9,HEINEKEN USA,97942,TECATE 4/6 LNNR - 12OZ,BEER,7.79,0.0,4.00
307643,2020,9,RELIABLE CHURCHILL LLLP,97950,S SMITH WINTER WELCOME NR 12/CS - 18.7OZ,BEER,0.00,0.0,2.00


In [4]:
# Revisar valores nulos
print(warehouse.isnull().sum())

# Imputar valores nulos si es necesario (aquí se imputan con 0 como ejemplo)
warehouse.fillna(0, inplace=True)

# Verificar nuevamente que no hay valores nulos
print(warehouse.isnull().sum())


YEAR                  0
MONTH                 0
SUPPLIER            167
ITEM CODE             0
ITEM DESCRIPTION      0
ITEM TYPE             1
RETAIL SALES          3
RETAIL TRANSFERS      0
WAREHOUSE SALES       0
dtype: int64
YEAR                0
MONTH               0
SUPPLIER            0
ITEM CODE           0
ITEM DESCRIPTION    0
ITEM TYPE           0
RETAIL SALES        0
RETAIL TRANSFERS    0
WAREHOUSE SALES     0
dtype: int64


In [8]:
# Agregado por proveedor (tienda)
agg_by_supplier = warehouse.groupby('SUPPLIER').agg({
    'RETAIL SALES': 'sum',
    'RETAIL TRANSFERS': 'sum',
    'WAREHOUSE SALES': 'sum'
}).reset_index()

print(agg_by_supplier.head())


                SUPPLIER  RETAIL SALES  RETAIL TRANSFERS  WAREHOUSE SALES
0                      0       7439.98            638.92        -20646.00
1             8 VINI INC          2.53              2.00             1.00
2        A HARDY USA LTD          0.56              0.00             0.00
3   A I G WINE & SPIRITS         13.24              4.92           197.00
4  A VINTNERS SELECTIONS       9482.87           8238.29         35241.97


In [9]:
# Agregado por artículo
agg_by_item = warehouse.groupby('ITEM CODE').agg({
    'RETAIL SALES': 'sum',
    'RETAIL TRANSFERS': 'sum',
    'WAREHOUSE SALES': 'sum'
}).reset_index()

print(agg_by_item.head())


  ITEM CODE  RETAIL SALES  RETAIL TRANSFERS  WAREHOUSE SALES
0    100002          0.17               0.0              0.0
1    100007          0.00               0.0              4.0
2    100008          0.00               0.0              3.0
3    100009          1.72               0.0             18.0
4    100011          0.00               0.0              3.0


In [12]:
# Conectar a la base de datos local (crea una nueva si no existe)
conn = sqlite3.connect('retail_sales.db')

# Escribir los datos limpios en la base de datos
warehouse.to_sql('cleaned_data', conn, if_exists='replace', index=False)

# Escribir los datos agregados por proveedor en la base de datos
agg_by_supplier.to_sql('aggregate_by_supplier', conn, if_exists='replace', index=False)

# Escribir los datos agregados por artículo en la base de datos
agg_by_item.to_sql('aggregate_by_item', conn, if_exists='replace', index=False)

# Confirmar cambios y cerrar la conexión
conn.commit()
conn.close()
